In [3]:
import os
import boto3
import sagemaker
from sagemaker.sklearn.model import SKLearnModel
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import CSVDeserializer


In [ ]:
os.environ["AWS_ACCESS_KEY_ID"]="Your AWS_ACCESS_KEY_ID"
os.environ["AWS_SECRET_ACCESS_KEY"]="Your AWS_SECRET_ACCESS_KEY"
os.environ["SAGEMAKER_ROLE"]="your SAGEMAKER_ROLE"

In [ ]:
AWS_ACCESS_KEY_ID = os.environ.get("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY")
region_name = "us-east-1"
role = os.environ.get("SAGEMAKER_ROLE")
AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, role

In [ ]:
boto_session = boto3.Session(region_name=region_name,
                             aws_access_key_id=AWS_ACCESS_KEY_ID,
                             aws_secret_access_key=AWS_SECRET_ACCESS_KEY)
sm_session = sagemaker.Session(boto_session=boto_session)
sm_client = boto_session.client("sagemaker")
bucket_name = "sagemaker07272025"
project_prefix = "amzn"
bucket_prefix = f"s3://{bucket_name}/{project_prefix}"
mlflow_artifact_path = f"s3://{bucket_prefix}/mlflow"


In [ ]:
a = sm_client.list_model_package_groups()
a

{'ModelPackageGroupSummaryList': [{'ModelPackageGroupName': 'testing-model-package-group',
   'ModelPackageGroupArn': 'arn:aws:sagemaker:us-east-1:395435558728:model-package-group/testing-model-package-group',
   'ModelPackageGroupDescription': 'This is group stores versions of my ML models.',
   'CreationTime': datetime.datetime(2025, 8, 10, 14, 41, 48, 955000, tzinfo=tzlocal()),
   'ModelPackageGroupStatus': 'Completed'}],
 'ResponseMetadata': {'RequestId': 'fa636ebf-b523-4cf6-b319-334627990f12',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'fa636ebf-b523-4cf6-b319-334627990f12',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '354',
   'date': 'Sun, 10 Aug 2025 11:07:29 GMT'},
  'RetryAttempts': 0}}

# Get model artifact from latest training

In [ ]:
job_name_prefix=f"{project_prefix}-sm-btd-preprocess"
# job_name_prefix
search_training = sm_client.search(
    Resource="TrainingJob",
    SearchExpression={
        "Filters": [
            {
                "Name": "TrainingJobName",
                "Operator": "Contains",
                'Value': job_name_prefix
            }
        ]
    },
    SortBy='CreationTime',
    SortOrder='Descending',
    MaxResults=1
)
sklearn_artifact_path = search_training["Results"][0]["TrainingJob"]["ModelArtifacts"]["S3ModelArtifacts"]

# Step 1 — Prepare Model Artifact for SageMaker

We package our trained model (`model.joblib`) along with the inference code into a `.tar.gz` file.
This archive will be uploaded to S3 and used by SageMaker for deployment.


In [ ]:
!tar -czvf model.tar.gz model.joblib code/
!aws s3 cp model.tar.gz s3://sagemaker07272025/manual/

'testing-model-package-group'

# Step 2 — SageMaker Session Setup

We set up AWS SDK (`boto3`) and SageMaker Python SDK sessions, along with our IAM execution role and model package group name.


In [ ]:
# Initialize SageMaker session and client
boto_session = boto3.Session(region_name=region_name,
                             aws_access_key_id=AWS_ACCESS_KEY_ID,
                             aws_secret_access_key=AWS_SECRET_ACCESS_KEY)
sm_session = sagemaker.Session(boto_session=boto_session)
sm_client = boto3.client("sagemaker")

# Your IAM execution role
role = "YOUR EXECUTION ROLE ARN"

# Model package group name (create if not exists)
model_package_group_name = "testing-model-package-group"
try:
    sm_client.create_model_package_group(
        ModelPackageGroupName=model_package_group_name,
        ModelPackageGroupDescription="Example group for local sklearn model"
    )
except sm_client.exceptions.ResourceInUse:
    print("Model package group already exists.")


# Step 3 — Create and Register Model

We create a `SKLearnModel` from our local artifact and register it in the SageMaker Model Registry.
Initially, we set status to `PendingManualApproval`.


In [ ]:
model_artifact = "s3://<your-bucket>/path/to/model.tar.gz"

model = SKLearnModel(
    model_data=model_artifact,
    role=role,
    entry_point="inference.py",
    source_dir="./sklearn_model/model/code",
    framework_version="1.2-1"
)

model_package = model.register(
    content_types=["text/csv"],
    response_types=["text/csv"],
    model_package_group_name=model_package_group_name,
    approval_status="PendingManualApproval",
    description="First registered version from local training"
)

print("Model package ARN:", model_package.model_package_arn)


In [ ]:
# We can also register the model with the model registry using below code as well
# model_status = "PendingManualApproval"
# response = sm_client.create_model_package(
#     ModelPackageGroupName=model_package_group_name,
#     ModelPackageDescription="First version of my model registered manually.",
#     InferenceSpecification={
#         "Containers": [
#             {
#                 "Image": image_uri[0],
#                 "ModelDataUrl": model_artifact
#             }
#         ],
#         "SupportedContentTypes": ["text/csv"],
#         "SupportedResponseMIMETypes": ["text/csv"],
#     },
#     ModelApprovalStatus=model_status
# )

# List model from the package group or registry

In [ ]:
mpg_res = sm_client.list_model_packages(
    ModelPackageGroupName=model_package_group_name
)

{'ModelPackageSummaryList': [{'ModelPackageGroupName': 'testing-model-package-group',
   'ModelPackageVersion': 8,
   'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:395435558728:model-package/testing-model-package-group/8',
   'ModelPackageDescription': 'First version of my model registered manually.',
   'CreationTime': datetime.datetime(2025, 8, 11, 18, 18, 46, 873000, tzinfo=tzlocal()),
   'ModelPackageStatus': 'Completed',
   'ModelApprovalStatus': 'PendingManualApproval'},
  {'ModelPackageGroupName': 'testing-model-package-group',
   'ModelPackageVersion': 7,
   'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:395435558728:model-package/testing-model-package-group/7',
   'ModelPackageDescription': 'First registered version from local training',
   'CreationTime': datetime.datetime(2025, 8, 11, 16, 38, 7, 247000, tzinfo=tzlocal()),
   'ModelPackageStatus': 'Completed',
   'ModelApprovalStatus': 'Approved'},
  {'ModelPackageGroupName': 'testing-model-package-group',
   'ModelPackage

# Step 4 — Approve Model for Deployment

SageMaker requires models from the registry to be `Approved` before deployment.
We update the model package status here.


In [ ]:
model_package_arn = mpg_res["ModelPackageSummaryList"][0]["ModelPackageArn"]

sm_client.update_model_package(
    ModelPackageArn=model_package_arn,
    ModelApprovalStatus="Approved",
    ApprovalDescription="Model tested and approved for deployment."
)


# Step 5 — Deploy the Approved Model

We create a SageMaker Model, Endpoint Configuration, and Endpoint from the approved model package.


In [ ]:
model_name = "testing-model-from-registry"
endpoint_config_name = "testing-model-endpoint-config"
endpoint_name = "testing-model-endpoint"

# Create Model
sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "ModelPackageName": model_package_arn
    }
)

# Create Endpoint Config
sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "AllTraffic",
            "ModelName": model_name,
            "InstanceType": "ml.m5.2xlarge",
            "InitialInstanceCount": 1
        }
    ]
)

# Create Endpoint
sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name
)


# Step 6 — Run Predictions

We use `sagemaker.predictor.Predictor` with CSV serialization/deserialization to send requests to the deployed endpoint.


In [ ]:
predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sm_session,
    serializer=CSVSerializer(),
    deserializer=CSVDeserializer()
)

payload = ["L,298.4,308.2,1582,70.7,216"]
prediction = predictor.predict(payload)
print("Prediction:", prediction)


# `inference.py` — Custom Inference Script

This script defines:
- `model_fn`: loads the model from disk
- `input_fn`: converts CSV input into a Pandas DataFrame with correct column names
- `predict_fn`: calls the model's `transform` method
- `output_fn`: returns predictions as CSV


In [ ]:
%%writefile sklearn_model/model/code/inference.py
import os
import joblib
import pandas as pd
import io

def model_fn(model_dir):
    model_path = os.path.join(model_dir, "model.joblib")
    print(f"Loading model from: {model_path}")
    return joblib.load(model_path)

def input_fn(input_data, content_type):
    if content_type == "text/csv":
        columns = ['Type', 'Air temperature [K]', 'Process temperature [K]',
                   'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]']
        df = pd.read_csv(io.StringIO(input_data), header=None)
        df.columns = columns
        return df
    else:
        raise ValueError(f"Unsupported content type: {content_type}")

def predict_fn(input_data, model):
    return model.transform(input_data)

def output_fn(prediction, accept):
    if accept == "text/csv":
        return ",".join(str(x) for x in prediction), accept
    else:
        raise ValueError(f"Unsupported accept type: {accept}")
